<font size="5"> <a href="https://uspdigital.usp.br/jupiterweb/obterDisciplina?sgldis=MAC0209"> MAC0209 - Modelagem e Simulação</a> </font>

Roberto M. Cesar Jr. (IME-USP)

Roberto Hirata Jr. (IME-USP)

Artur André A.M. Oliveira (IME-USP)

***
<font size="5"> Análise de dados de mobilidade usando dados do Kartaview: Parte 1 </font>
***

# Setup e Lib

In [ ]:
!python -m pip install pyproj

from google.colab import drive
drive.mount('/content/drive')

# edite seu path abaixo:

workDir = '/content/drive/MyDrive/doc/Courses/Modelagem/programas/jupyter/2022/kartaview/'


In [ ]:
import matplotlib.pyplot as plt

from datetime import datetime
import json

import numpy as np
import matplotlib.pyplot as plt

from pyproj import Proj, transform

import argparse
import re


In [ ]:
# arquivos uteis usados no JN abaixo

arquivoBase = "sample1.json"
arquivoBase = "sample3.json"
arquivoBase = "kartaview-pracaPanamericana.json"

arquivo_json = workDir + arquivoBase

jsonfileName = arquivoBase
jsonfile = workDir + jsonfileName
extracted_filename = workDir + "extracted_" + jsonfileName
cleaned_filename = workDir + "cleaned_" + jsonfileName

arquivo_pontos = cleaned_filename


# O formato JSON


Referência: [json.org](https://www.json.org/json-en.html)

JSON é uma notação usada para criar mensagens de texto estruturadas. A ideia/motivação principal é que sejam simples de serem lidas por seres humanos e ao mesmo tempo fáceis de serem geradas/lidas por um programa (parser).

Um mensagem no formato JSON conta com duas estruturas básicas: (1) uma coleção de pares chave/valor delimitados por chaves '{}' (similar à dicionários em python); e (2) uma sequência de elementos, que são delimitados por colchetes '[]' (similar à listas em python).

Uma coleção de pares chave/valor é chamada de objeto, a chave em cada par chave/valor deve ser uma string e um objeto não pode ter duas chaves iguais.

Uma sequência não possui chaves e seus elementos podem ser mistos (por exemplo: outras sequências, objetos e elementos).

Um elemento pode ser ou uma string delimitada por aspas duplas "exemplo de string" ou um número.

Todos os exemplos a seguir são mensagens válidas de acordo com o formato JSON:

1. ""
2. "JSON válido"
3. []
4. ["A sequancia anterior está vazia", "Já esta aqui tem duas strings e um número", 7]
5. {"chave": "valor da chave para este objeto"}
6. {}
7. "A linha anterior tem um objeto vazio"

Apesar da notação admitir mensagens com apenas uma string, números, ou sequência, muitas vezes aplicações que dão suporte ao formato JSON não admitem mensagens que não contenham um objeto principal, isso é, todos os elementos da mensagem são encapsulados num objeto raiz, por exemplo:

{
  "c1": {"chave nutela": "objeto interno", "id" : 7},
  "outra chave": 2,
  "alguma sequencia importante": [2,3,5,7,11]
}

Note que além de termos um objeto principal na mensagem anterior (delimitado pelas chaves na primeira e ultima linha) temos outro objeto no primeiro par chave/valor (com a chave "c1"). Cada par chave/valor num objeto deve ser separado por vírgulas de outros pares no mesmo objeto.

Existem diversos [editores online de JSON](https://jsoneditoronline.org/#left=local.nijuma&right=local.pisura) que podemos usar para explorar mensagens JSON rapidamente, ou para entender como o formato funciona.

# Carregando o arquivo de dados do KartaView

Vamos agora analisar os dados coletados através da plataforma KartaView, como explicado em aula. 

Esses dados estão formatados usando-se o formato JSON de JavaScript Object Notation.

In [ ]:
# arquivo_json = workDir + "sample1-Ori.json"
# arquivo_json = workDir + "sample1.json"
# arquivo_json = workDir + "sample3.json"

with open(arquivo_json, "r") as f:
    pontos = f.read()
    pontos = json.loads(pontos)

In [ ]:
print(f"Chaves na raiz - \n{pontos.keys()}\n")
print(f"Chaves do objeto na chave 'status' - \n{pontos['status'].keys()}\n")
print(f"Chaves do objeto na chave 'osv' - \n{pontos['osv'].keys()}")

Seguindo a sequencia de chaves 'osv' -> 'photos' chegamos à uma sequência de objetos.

Cada um destes objetos representa os metadados de uma imagem tomada em um ponto do trajeto escolhido no KartaView.

In [ ]:
# Exemplo de um objeto na chave 'photos':
pontos['osv']['photos'][0]


Podemos observar que muitos dos campos parecem ter utilidade apenas para a aplicação do KartaView.

Para simplificar a análise vamos criar um novo arquivo JSON (chamado extracted que contenha um único objeto raiz com um único par chave/valor. Este par será o campo 'photos' e a sequência para a qual ele é chave:

`{
"photos" :
    [
    {
        'id': '206416139',
         'sequence_id': '1131993',
         'sequence_index': '0',
         'lat': '32.188423',
         'lng': '-81.195239',
         'fileName': '1131993_119db_1.jpg',
         'name': 'storage7/files/photo/2018/3/4/proc/1131993_119db_1.jpg',
         'lth_name': 'storage7/files/photo/2018/3/4/lth/1131993_119db_1.jpg',
         'th_name': 'storage7/files/photo/2018/3/4/th/1131993_119db_1.jpg',
         'path': '2018/3/4',
         ...
     },
     ...
    ]
}`


### Exercício: limpeza e simplificação do JSON

Usando o arquivo 'extracted_sample1.json' gerado na seção anterior, execute a célula anterior e crie um novo arquivo JSON chamado 'cleaned_sample1.json' em que cada objeto da sequência 'photos' contém somente os campos:

- 'lat'
- 'lng'
- 'heading'
- 'shot_date'

Ou seja, o novo arquivo gerado a partir do 'extracted_sample1.json' deverá seguir o modelo:

`{
"photos" :
    [
    {
        'lat': '32.188423',
        'lng' : '-81.195239',
        'heading' : '72.76266',
        'shot_date': '2018-03-03 20:29:36'
     },
     ...
    ]
}`

### Solução do professor: só olhe depois de programar a sua

In [ ]:
# funcoes uteis para preparar limpar e simplificar o json
# tente primeiro criar as suas funcoes. Depois, compare.

def get_photo_array_positions(txt):
  s = txt.index("\"photos\":[{")   
  e = txt.index("}]", s + 1)
  return s, e + len("}]")

def clean_extracted(txt):
    patterns = [r'"fileName":".*?",',
        r'"name":".*?",',
        r'"lth_name":".*?",',
        r'"th_name":".*?",',
        r'"path":".*?",',
        r'"way_id":".*?",',
        r'"id":".*?",',
        r'"sequence_id":".*?",',
        r'"sequence_index":".*?",',
        r'"timestamp":".*?",',
        r'"storage":".*?",',
        r'"field_of_view":null,',
        r'"is_unwrapped":".*?",',
        r'"unwrap_version":".*?",',
        r'"width":".*?",',
        r'"height":".*?",',
        r'"projection":".*?",',
        r'"cameraParameters":null,',
        r'"hasDepth":false',
        r'"hasDepth":true'
    ]
    clean = txt
    for pattern in patterns:
        m = re.search(pattern, txt)
        if m is not None:
            print(m)
        clean = re.sub(pattern, '', clean)
    clean = re.sub(",},", '},\n', clean)
    clean = re.sub('",}]}', '"}]}', clean)
    
    return clean

def make_extract_photos_JSON(output_file, json_msg):
    first_pos, last_pos = get_photo_array_positions(json_msg)
    extracted_str = "{" + json_msg[first_pos:last_pos] + "}"
    with open(output_file, "w") as ejf:
        ejf.write(extracted_str)
    return extracted_str

### Solução do aluno: programe a sua aqui

In [ ]:
# funcoes uteis para preparar limpar e simplificar o json
# tente primeiro criar as suas funcoes. Depois, compare.

#def get_photo_array_positions(txt):
 

#def clean_extracted(txt):


#def make_extract_photos_JSON(output_file, json_msg):


In [ ]:
# esta celula usa as funcoes da celula anterior para gerar um JSON simplificado e limpo

# jsonfileName = 'sample3.json'
# jsonfile = workDir + jsonfileName
# extracted_filename = workDir + "extracted_" + jsonfileName
# cleaned_filename = workDir + "cleaned_" + jsonfileName

with open(jsonfile, "r") as jf:
        # Aqui o array 'photos' é extraído e colocado 
        # em outro arquivo (com o prefixo extracted_),
        # para facilitar o processamento do array.
        #
        # Em seguida usamos a função clean_extracted para
        # criar um terceiro arquivo (com prefixo _cleaned)
        # que contenha somente os campos de interesse para
        # a análise.
        txt = jf.read()
        txt = re.sub("\s", "", txt)
        extracted_str = make_extract_photos_JSON(extracted_filename, txt)

        with open(cleaned_filename, "w") as cjf:
           cjf.write(clean_extracted(extracted_str))


# Definindo uma faixa de pontos

O trajeto pode conter muitos pontos (e.g. 7800+ em um dos JSON de exemplo). Para facilitar a nossa vida e a análise, vamos selecionar um conjunto menor de pontos para este exercício. 

In [ ]:
# Vamos carregar os pontos (do JSON filtrado) na variável pontos.
# arquivo_pontos = workDir+"cleaned_sample1.json"

with open(arquivo_pontos, "r") as f:
    pontos = f.read()
    pontos = json.loads(pontos)
    pontos = pontos['photos']
    print('Tamanho da amostra: ', len(pontos))



In [ ]:
# Aqui escolhemos quais pontos iremos usar no intervalo entre [0-len(pontos])

# abaixo, uma selecao de procao intermediaria
n2 = int(len(pontos)/2)
intervalo = 500
i0 = n2-intervalo
i1 = n2+intervalo

# abaixo, uma selecao arbitraria
i0 = 0
i1 = 1000

faixa_de_pontos = range(i0,i1)

In [ ]:
# Um ponto da rota escolhida é um objeto com as seguintes propriedades
print(pontos[faixa_de_pontos[0]])

x = [float(pontos[i]['lng']) for i in faixa_de_pontos]
x = np.asarray(x)

y = [float(pontos[i]['lat']) for i in faixa_de_pontos]
y = np.asarray(y)

plt.plot(x,y)
plt.axis('equal')
plt.show()


# Medindo distâncias

## Funções auxiliares

Para facilitar um pouco a escrita do código e termos um código focado
no problema que queremos resolver, ao invés de nos preocuparmos com
detalhes da estrutura interna dos dados, vamos definir aqui algumas
funções auxiliares com a responsabilidade exclusiva de coletar uma propriedade específica da lista de pontos, possivelmente tratando o dado coletado.

### def get_point_coords(index, points_object)

Essa função irá nos auxiliar para coletar as coordenadas (e.g. longitude e latitude) de um ponto (na posição 'index') da lista de pontos passado também como parâmetro da função.

Note que as coordenadas dos pontos (no arquivo de dados) usam a projeção EPSG:4326, isso significa que estas coordenadas são ângulos e portanto precisamos fazer uma conversão, ou mais precisamente uma (re)projeção em um outro sistema de coordenadas (i.e. CRS) que use unidades métricas (e.g. metros).

In [ ]:
def get_point_coords(index, points_object):
    """
    Essa função recebe um índice numérico correspondendo a uma
    posição na lista de pontos "points_object".
    
    Ela retorna um vetor do numpy com a longitude e latitude
    (propriedades 'lng' e 'lat') do ponto na posição 'index'.
    """
    lat = points_object[index]['lat']
    lat = float(lat)
    lng = points_object[index]['lng']
    lng = float(lng)
    return np.array((lng, lat))


### def get_point_coords_proj(index, points_object)

Essa função auxiliar faz exatamente o mesmo que a anterior, contudo os pontos aqui são reprojetados para a projeção EPSG:3857, que usa como unidade métrica o 'metro' ao invés de graus de ângulo.

In [ ]:
def get_point_coords_proj(index, points_object):
    """
    Essa função é similar a get_point_coords, ela 
    recebe um índice numérico correspondendo a uma
    posição na lista de pontos "points_object".
    
    Contudo esta os pontos na projeção EPSG:3857 em
    que a unidade de medida é em metros e portanto
    podemos calcular a distância euclidiana entre dois
    pontos com base em suas coordenadas.
    
    Os pontos retornados são um vetor numpy em que
    a primeira posição é uma medida em metros no eixo
    horizontal e a segunda é num eixo vertical.
    O ponto de origem pode ser visto aqui https://epsg.io/3857
    """
    
    lat = points_object[index]['lat']
    lat = float(lat)
    lng = points_object[index]['lng']
    lng = float(lng)
    p = np.array((lng, lat))
    p = transform(Proj(init='epsg:4326'), Proj(init='epsg:3857'), p[0], p[1])
    return p

### def get_shot_time(index, points_object)

Esta função é um acessor para a propriedade 'shot_date' na lista de pontos. Essa propriedade indica o momento (dia e hora incluindo segundos) em que o ponto foi criado.

In [ ]:
FMT = '%Y-%m-%d %H:%M:%S'
def get_shot_time(index, points_object):
    """
    Retorna a data e hora em que o ponto 'index',
    da lista de pontos 'points_object', foi criado.
    
    O formato de retorno é uma string '%Y-%m-%d %H:%M:%S'
    (e.g. 2018-03-03 20:55:32)
    """
    return points_object[index]['shot_date']

### Visualizando a distância percorrida no tempo
Note que o movimento não é perfeitamente uniforme em todos os momentos e note também que nem todos os momentos de tempo existem no conjunto de dados (e.g. lacunas entre dois 'segmentos de reta')

## Exemplo com 2 pontos

Aqui vamos medir distâncias entre dois pontos
para termos uma ideia de como usar as definições
e funções auxiliares definidas até aqui.

### Coordenadas em metros

Usamos a função auxiliar get_point_coords_proj, cujas coordenadas
de retorno usam unidades de medida em metros.

In [ ]:
pp1 = get_point_coords_proj(faixa_de_pontos[0], pontos)

pp2 = get_point_coords_proj(faixa_de_pontos[1], pontos)

# Note que aqui temos uma unidade de metros à leste
# (ou oeste em caso negativo) e à norte (ou sul
# em caso negativo) de um dado local no planeta.

print(f'easting pp1: {pp1[0]} m; northing pp1: {pp1[1]} m')
print(f'easting pp2: {pp2[0]} m; northing pp2: {pp2[1]} m')

## Exercícios

### Medindo a distância entre dois pontos

Neste exercício você deve implementar a função distancia_euclidiana.

Esta função recebe dois vetores (i.e. array numpy) cada um com duas dimensões e retorna
a distância euclidiana entre eles.

In [ ]:
def distancia_euclidiana(v1, v2):
    """
    Tendo como parâmetros os vetores numpy 2D 'v1' e 'v2', crie uma
    função que retorne a distância euclidiana entre os dois vetores.
    """

    return(0)

In [ ]:
# Desta vez a distância resultante pode ser interpretada
# fisicamente em metros.

print(f'A distância entre pp1 e pp2 é: {distancia_euclidiana(pp1, pp2)} metros.')

In [ ]:
# Teste artificial

pv1 = np.array([1, 1]) # Ponto x = 1, y = 1
pv2 = np.array([1, -1]) # Ponto x = 2, y = 0

print(f'A distância entre pv1 e pv2 é 2')
print(f'A distância retornada pela função implementada foi {distancia_euclidiana(pv1, pv2)}\n')
if (distancia_euclidiana(pv1, pv2) == 2):
    print("A distância foi calculada corretamente")
else:
    print("A distância foi calculada incorretamente\n")
    print(f"O erro entre o esperado e o calculado foi {2 - distancia_euclidiana(pv1, pv2)}")

### Exercício - Visualizando o comprimento de cada trecho

Usando a função criada no exercício anterior vamos agora ver a distância percorrida pelo veículo a cada passo do sub-trajeto selecionado:

A ideia aqui é que todos os pontos da faixa de pontos selecionada sejam percorridos, e para cada par de pontos subsequentes seja calculada a distância entre eles e impressa essa distância.

Imprima uma lista como o exemplo a seguir usando a sua função de medir distância entre dois pontos:


- trecho 1000 a 1001 ; d = 0
- trecho 1001 a 1002 ; d = 0
- trecho 1002 a 1003 ; d = 0
- trecho 1003 a 1004 ; d = 0
- ...
- trecho 1097 a 1098 ; d = 0
- trecho 1098 a 1099 ; d = 0
- trecho 1099 a 1100 ; d = 0

Dica, você pode usar a linha abaixo para imprimir uma vez que a distância foi calculada:

`print(f'trecho {i} {i+1} ; d = {dist}')`